In [79]:
import pandas as pd
import plotly.express as px
import numpy as np

In [29]:
df_bm_units = pd.read_excel("../data/BMUFuelType.xls", header=0)

In [136]:
df_fpn = pd.read_csv(
    "../data/PhysicalData_20220319_27.csv",
    skiprows=lambda x: x > 1541 or x == 0,
    names=[
        "Notification Type",
        "Unit ID",
        "Settlement Period",
        "From Time",
        "From Level",
        "To Time",
        "To Level",
    ],
    parse_dates=["From Time", "To Time"],
    date_parser=lambda x: pd.to_datetime(x, format="%Y%m%d%H%M%S"),
)  # This includes other kinds of data with different cols

df_fpn = (
    df_fpn.set_index("Unit ID")
    .join(df_bm_units.set_index("SETT_BMU_ID")["FUEL TYPE"])
    .rename(columns={"FUEL TYPE": "Fuel Type"})
)

There are many different forms here:
- FPN (Final Physical Notification): what the operators tell the grid they're going to generate
- QPN (Quiescent Physical Notifications)
- MEL (Maximum Export Limit)
- MIL (Maximum Import Limit)
- BOAL (Bid-Offer Acceptance Level Data)

In [137]:
df_boal = pd.read_csv(
    "../data/PhysicalData_20220319_27.csv",
    skiprows=lambda x: x < 5718,
    names=[
        "Notification Type",
        "Unit ID",
        "Accept ID",
        "Accept Time",
        "Deemed Flag",
        "SO Flag",
        "STOR Provider Flag",
        "RR Instruction Flag",
        "RR Schedule Flag",
        "From Time",
        "From Level",
        "To Time",
        "To Level",
    ],
    parse_dates=["Accept Time", "From Time", "To Time"],
    date_parser=lambda x: pd.to_datetime(x, format="%Y%m%d%H%M%S"),
)  # This includes other kinds of data with different cols

df_boal = df_boal[df_boal["Notification Type"] == "BOALF"]
df_boal['Accept ID'] = df_boal['Accept ID'].astype(int)
df_boal = (
    df_boal.set_index("Unit ID")
    .join(df_bm_units.set_index("SETT_BMU_ID")["FUEL TYPE"])
    .rename(columns={"FUEL TYPE": "Fuel Type"})
)

In [138]:
BOOL_COLS = [
    "Deemed Flag",
    "SO Flag",
    "STOR Provider Flag",
    "RR Instruction Flag",
    "RR Schedule Flag",
]

df_boal[BOOL_COLS] = df_boal[BOOL_COLS].apply(lambda x: x == "T")

In [139]:
df_boal[df_boal["SO Flag"]]

,Notification Type,Accept ID,Accept Time,Deemed Flag,SO Flag,STOR Provider Flag,RR Instruction Flag,RR Schedule Flag,From Time,From Level,To Time,To Level,Fuel Type
T_CARR-1,BOALF,79613,2022-03-19 12:09:00,False,True,False,False,False,2022-03-19 13:00:00,176.0,2022-03-19 13:30:00,176.0,CCGT
T_CARR-1,BOALF,79614,2022-03-19 12:44:00,False,True,False,False,False,2022-03-19 12:46:00,176.0,2022-03-19 13:46:00,176.0,CCGT
T_CARR-1,BOALF,79615,2022-03-19 13:26:00,False,True,False,False,False,2022-03-19 13:28:00,176.0,2022-03-19 14:13:00,176.0,CCGT
T_EAAO-1,BOALF,4379,2022-03-19 12:46:00,False,True,False,False,False,2022-03-19 12:49:00,230.0,2022-03-19 13:19:00,230.0,WIND
T_EAAO-1,BOALF,4379,2022-03-19 12:46:00,False,True,False,False,False,2022-03-19 13:19:00,230.0,2022-03-19 13:22:00,336.0,WIND
T_EAAO-1,BOALF,4380,2022-03-19 13:04:00,False,True,False,False,False,2022-03-19 13:19:00,230.0,2022-03-19 13:49:00,230.0,WIND
T_LAGA-1,BOALF,133192,2022-03-19 12:00:00,False,True,False,False,False,2022-03-19 13:20:00,0.0,2022-03-19 13:30:00,17.0,CCGT
T_LAGA-1,BOALF,133193,2022-03-19 12:32:00,False,True,False,False,False,2022-03-19 13:29:00,16.0,2022-03-19 14:00:00,71.0,CCGT
T_MRWD-1,BOALF,144962,2022-03-19 12:01:00,False,True,False,False,False,2022-03-19 12:59:00,450.0,2022-03-19 13:30:00,450.0,CCGT
T_MRWD-1,BOALF,144965,2022-03-19 12:32:00,False,True,False,False,False,2022-03-19 13:29:00,450.0,2022-03-19 14:00:00,450.0,CCGT


In [140]:
df_boal_wind = df_boal[df_boal["Fuel Type"] == "WIND"]

In [141]:
df_boal_wind

,Notification Type,Accept ID,Accept Time,Deemed Flag,SO Flag,STOR Provider Flag,RR Instruction Flag,RR Schedule Flag,From Time,From Level,To Time,To Level,Fuel Type
E_BTUIW-3,BOALF,3642,2022-03-19 12:35:00,False,False,False,False,False,2022-03-19 13:20:00,0.0,2022-03-19 13:22:00,43.0,WIND
E_BTUIW-3,BOALF,3643,2022-03-19 13:05:00,False,False,False,False,False,2022-03-19 13:20:00,0.0,2022-03-19 13:50:00,0.0,WIND
T_AKGLW-3,BOALF,2005,2022-03-19 12:26:00,False,False,False,False,False,2022-03-19 12:58:00,0.0,2022-03-19 13:03:00,48.0,WIND
T_AKGLW-3,BOALF,2006,2022-03-19 12:50:00,False,False,False,False,False,2022-03-19 12:52:00,0.0,2022-03-19 13:52:00,0.0,WIND
T_AKGLW-3,BOALF,2007,2022-03-19 13:02:00,False,False,False,False,False,2022-03-19 13:04:00,0.0,2022-03-19 14:19:00,0.0,WIND
...,...,...,...,...,...,...,...,...,...,...,...,...,...
T_WLNYO-3,BOALF,22300,2022-03-19 13:07:00,False,False,False,False,False,2022-03-19 13:17:00,0.0,2022-03-19 13:37:00,0.0,WIND
T_WLNYO-4,BOALF,23054,2022-03-19 12:30:00,False,False,False,False,False,2022-03-19 13:00:00,0.0,2022-03-19 13:06:00,253.0,WIND
T_WLNYO-4,BOALF,23055,2022-03-19 12:50:00,False,False,False,False,False,2022-03-19 13:00:00,0.0,2022-03-19 13:20:00,0.0,WIND
T_WLNYO-4,BOALF,23055,2022-03-19 12:50:00,False,False,False,False,False,2022-03-19 13:20:00,0.0,2022-03-19 13:26:00,262.0,WIND


Curtailment is if the BOALF level is less than the FPN

In [142]:
unit = "T_EAAO-1"
df_boal_wind.loc[unit]

,Notification Type,Accept ID,Accept Time,Deemed Flag,SO Flag,STOR Provider Flag,RR Instruction Flag,RR Schedule Flag,From Time,From Level,To Time,To Level,Fuel Type
T_EAAO-1,BOALF,4379,2022-03-19 12:46:00,False,True,False,False,False,2022-03-19 12:49:00,230.0,2022-03-19 13:19:00,230.0,WIND
T_EAAO-1,BOALF,4379,2022-03-19 12:46:00,False,True,False,False,False,2022-03-19 13:19:00,230.0,2022-03-19 13:22:00,336.0,WIND
T_EAAO-1,BOALF,4380,2022-03-19 13:04:00,False,True,False,False,False,2022-03-19 13:19:00,230.0,2022-03-19 13:49:00,230.0,WIND
T_EAAO-1,BOALF,4381,2022-03-19 13:19:00,False,False,False,False,False,2022-03-19 13:21:00,230.0,2022-03-19 13:22:00,220.0,WIND
T_EAAO-1,BOALF,4381,2022-03-19 13:19:00,False,False,False,False,False,2022-03-19 13:22:00,220.0,2022-03-19 13:52:00,220.0,WIND


In [143]:
df_fpn.loc[unit]

Notification Type                     PN
Settlement Period                     27
From Time            2022-03-19 13:00:00
From Level                           336
To Time              2022-03-19 13:30:00
To Level                             336
Fuel Type                           WIND
Name: T_EAAO-1, dtype: object

This is a curtailed unit! 

In [179]:
def linearize_physical_data(df: pd.DataFrame):

    from_columns = ["From Level", "From Time"]
    to_columns = ["To Level", "To Time"]

    base_columns = [x for x in df.columns.copy() if x not in from_columns + to_columns]

    df = pd.concat(
        (
            df[base_columns + from_columns].rename(
                columns={"From Level": "Level", "From Time": "Time"}
            ),
            df[base_columns + to_columns].rename(
                columns={"To Level": "Level", "To Time": "Time"}
            ),
        )
    )

    df['Level'] = df['Level'].astype(float)
    return df

In [145]:
df_boal_linear = linearize_physical_data(df_boal.loc[unit])
df_boal_linear["Accept Time str"] = df_boal_linear["Accept Time"].astype(str)

px.line(df_boal_linear, "Time", "Level", color="Accept Time str")

Complexity here: the offers change over time, so I think you need to take the level indicated by the latest bid offer at every time pointm

In [146]:
df_boal_linear

,Notification Type,Accept ID,Accept Time,Deemed Flag,SO Flag,STOR Provider Flag,RR Instruction Flag,RR Schedule Flag,Fuel Type,Level,Time,Accept Time str
T_EAAO-1,BOALF,4379,2022-03-19 12:46:00,False,True,False,False,False,WIND,230.0,2022-03-19 12:49:00,2022-03-19 12:46:00
T_EAAO-1,BOALF,4379,2022-03-19 12:46:00,False,True,False,False,False,WIND,230.0,2022-03-19 13:19:00,2022-03-19 12:46:00
T_EAAO-1,BOALF,4380,2022-03-19 13:04:00,False,True,False,False,False,WIND,230.0,2022-03-19 13:19:00,2022-03-19 13:04:00
T_EAAO-1,BOALF,4381,2022-03-19 13:19:00,False,False,False,False,False,WIND,230.0,2022-03-19 13:21:00,2022-03-19 13:19:00
T_EAAO-1,BOALF,4381,2022-03-19 13:19:00,False,False,False,False,False,WIND,220.0,2022-03-19 13:22:00,2022-03-19 13:19:00
T_EAAO-1,BOALF,4379,2022-03-19 12:46:00,False,True,False,False,False,WIND,230.0,2022-03-19 13:19:00,2022-03-19 12:46:00
T_EAAO-1,BOALF,4379,2022-03-19 12:46:00,False,True,False,False,False,WIND,336.0,2022-03-19 13:22:00,2022-03-19 12:46:00
T_EAAO-1,BOALF,4380,2022-03-19 13:04:00,False,True,False,False,False,WIND,230.0,2022-03-19 13:49:00,2022-03-19 13:04:00
T_EAAO-1,BOALF,4381,2022-03-19 13:19:00,False,False,False,False,False,WIND,220.0,2022-03-19 13:22:00,2022-03-19 13:19:00
T_EAAO-1,BOALF,4381,2022-03-19 13:19:00,False,False,False,False,False,WIND,220.0,2022-03-19 13:52:00,2022-03-19 13:19:00


In [147]:
df_boal_linear.reset_index().rename(columns={"index": "Unit"})

,Unit,Notification Type,Accept ID,Accept Time,Deemed Flag,SO Flag,STOR Provider Flag,RR Instruction Flag,RR Schedule Flag,Fuel Type,Level,Time,Accept Time str
0,T_EAAO-1,BOALF,4379,2022-03-19 12:46:00,False,True,False,False,False,WIND,230.0,2022-03-19 12:49:00,2022-03-19 12:46:00
1,T_EAAO-1,BOALF,4379,2022-03-19 12:46:00,False,True,False,False,False,WIND,230.0,2022-03-19 13:19:00,2022-03-19 12:46:00
2,T_EAAO-1,BOALF,4380,2022-03-19 13:04:00,False,True,False,False,False,WIND,230.0,2022-03-19 13:19:00,2022-03-19 13:04:00
3,T_EAAO-1,BOALF,4381,2022-03-19 13:19:00,False,False,False,False,False,WIND,230.0,2022-03-19 13:21:00,2022-03-19 13:19:00
4,T_EAAO-1,BOALF,4381,2022-03-19 13:19:00,False,False,False,False,False,WIND,220.0,2022-03-19 13:22:00,2022-03-19 13:19:00
5,T_EAAO-1,BOALF,4379,2022-03-19 12:46:00,False,True,False,False,False,WIND,230.0,2022-03-19 13:19:00,2022-03-19 12:46:00
6,T_EAAO-1,BOALF,4379,2022-03-19 12:46:00,False,True,False,False,False,WIND,336.0,2022-03-19 13:22:00,2022-03-19 12:46:00
7,T_EAAO-1,BOALF,4380,2022-03-19 13:04:00,False,True,False,False,False,WIND,230.0,2022-03-19 13:49:00,2022-03-19 13:04:00
8,T_EAAO-1,BOALF,4381,2022-03-19 13:19:00,False,False,False,False,False,WIND,220.0,2022-03-19 13:22:00,2022-03-19 13:19:00
9,T_EAAO-1,BOALF,4381,2022-03-19 13:19:00,False,False,False,False,False,WIND,220.0,2022-03-19 13:52:00,2022-03-19 13:19:00


You need to resample to a high frequency for each overlapping acceptance period separately and then recombine

In [160]:
out = []
for accept_id, data in df_boal_linear.groupby("Accept ID"):

    high_freq = (
        data.reset_index()
        .rename(columns={"index": "Unit"})
        .set_index("Time")
        .resample("1T")
        .first()

    )
    out.append(high_freq.interpolate().fillna(method='ffill'))

In [162]:
recombined = pd.concat(out)

In [168]:
px.line(recombined, x=recombined.index, y ="Level", color='Accept ID', hover_data=['Accept Time'])

In [164]:
resolved = recombined.reset_index().groupby('Time').last()

In [178]:
linearize_physical_data(pd.DataFrame(df_fpn.loc[unit]).T).dtypes

Notification Type            object
Settlement Period            object
Fuel Type                    object
Level                        object
Time                 datetime64[ns]
dtype: object

In [182]:
unit_fpn_resolved = linearize_physical_data(pd.DataFrame(df_fpn.loc[unit]).T).set_index('Time').resample('1T').mean().interpolate()

In [183]:
unit_fpn_resolved.head()

,Level
Time,
2022-03-19 13:00:00,336.0
2022-03-19 13:01:00,336.0
2022-03-19 13:02:00,336.0
2022-03-19 13:03:00,336.0
2022-03-19 13:04:00,336.0


In [187]:
unit_fpn_resolved["Notification Type"] = 'FPN'
combined = pd.concat((resolved, unit_fpn_resolved))

In [170]:
fig = px.line(resolved, x=resolved.index, y ="Level", color='Accept ID')
fig.add_hline(336) # This is what the unit SAID it would do

In [188]:
px.line(combined, x=combined.index, y='Level', color='Notification Type')